In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np


In [2]:
# ------------------ #
# IMPORT DES DONNÉES #
# ------------------ #
ENTREPOT_PATH = '~/Bureau/Datagrosyst/data_entrepot_&_outils/'
EXTERN_DATA_PATH = '/home/administrateur/Bureau/doc_de_travail/Indicateurs de la Cellule Ref/Data_cell_ref/'
df = {}

def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False).replace({'\r\n': '\n'}, regex=True)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(' - ', df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
    'espece', 'variete', 'culture', 'composant_culture'
]

extern_data = [
    'typo_especes_typo_culture', 'typo_especes_typo_culture_marai', 'espece_standard_typo_espece'
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',', index_col='id', verbose=False)

# import des données du magasin
import_dfs(extern_data, EXTERN_DATA_PATH, sep = ',', verbose=False)

100%|██████████| 3/3 [00:00<00:00, 521.94it/s]


In [3]:
esp = df['espece']
esp = esp[['libelle_espece_botanique','libelle_qualifiant_aee','libelle_type_saisonnier_aee','libelle_destination_aee']]

In [4]:
esp.loc[:, 'nom_espece_standard'] = np.select(
	[
		esp['libelle_espece_botanique'] == 'Pois',
		esp['libelle_espece_botanique'].isin(['Betterave','Chou','Concombre','Haricot','Lin','Pomme de terre','Prairie','Céleri']),
		esp['libelle_espece_botanique'].isin(['Avoine','Blé dur','Blé tendre','Carotte','Chanvre','Chicorée','Colza','Dactyle',
									  'Epeautre','Fenugrec','Fétuque des prés','Fétuque élevée','Féverole','Fléole des prés',
									  'Gesse Commune ou pois carré','Lentille','Lotier','Lotier corniculé','Lupin blanc','Luzerne',
									  'Millet commun','Orge','Pois chiche','Ray-grass anglais',"Ray-grass d'Italie",'Ray-grass hybride',
									  'Sainfoin','Salsifis','Sarrasin','Seigle','Soja','Sorgho','Tournesol','Trèfle blanc',
									  "Trèfle d'Alexandrie",'Trèfle hybride','Trèfle incarnat','Trèfle violet','Triticale','Vesce commune']),
		((esp['libelle_espece_botanique'] == 'Radis') & (esp['libelle_qualifiant_aee'] == 'Fourrager / Fourrage')),
		((esp['libelle_espece_botanique'] == 'Oignon') & (esp['libelle_qualifiant_aee'].isna())),
		((esp['libelle_espece_botanique'] == 'Oignon') & (esp['libelle_qualifiant_aee'].notna()))
	], 
	[
		esp[['libelle_espece_botanique','libelle_qualifiant_aee','libelle_type_saisonnier_aee']].fillna('').apply(lambda row: ' '.join(row.values.astype(str)).strip(), axis=1),
		esp[['libelle_espece_botanique','libelle_qualifiant_aee']].fillna('').apply(lambda row: ' '.join(row.values.astype(str)).strip(), axis=1),
		esp[['libelle_espece_botanique','libelle_type_saisonnier_aee']].fillna('').apply(lambda row: ' '.join(row.values.astype(str)).strip(), axis=1),
		esp[['libelle_espece_botanique','libelle_qualifiant_aee']].fillna('').apply(lambda row: ' '.join(row.values.astype(str)).strip(), axis=1),
		esp[['libelle_espece_botanique','libelle_type_saisonnier_aee']].fillna('').apply(lambda row: ' '.join(row.values.astype(str)).strip(), axis=1),
		esp[['libelle_espece_botanique','libelle_qualifiant_aee']].fillna('').apply(lambda row: ' '.join(row.values.astype(str)).strip(), axis=1)
	]
)

esp.loc[:, 'nom_espece_standard'] = np.where(esp['nom_espece_standard'] == 0, esp['libelle_espece_botanique'], esp['nom_espece_standard'])

/tmp/ipykernel_86841/3217965752.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  esp.loc[:, 'nom_espece_standard'] = np.select(


In [5]:
espece = esp.reset_index().merge(df['espece_standard_typo_espece'], on='nom_espece_standard', how="left").set_index('id')
espece = espece[['Typo_espece','Typo_espece_2']].rename(columns={'Typo_espece': 'typocan_espece', 'Typo_espece_2': 'typocan_espece_maraich'})
# espece.to_csv(EXTERN_DATA_PATH + 'refespece_supp_col_can.csv')

In [6]:
# df['espece'].merge(espece, how='left', on='id').to_csv(ENTREPOT_PATH + 'espece_vCAN.csv')